### CONFIGURATION

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np
import copy
import sys
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Hartwig imports
sys.path.append("/Users/peterpriestley/hmf/repos/scripts/analysisscripts") 
import analyseVCF as aVCF
import venn as vn

In [3]:
import os
VCF_PATH = "/Users/peterpriestley/hmf/analyses/cancerSample/"
VCF_SAMPLES=[]
#VCF_SAMPLES = ["CPCT02010269","CPCT02010299"]
#Find all VCF FILES in VCF directory
for x in os.listdir(VCF_PATH):
    if x[-3:] == "vcf":
        VCF_SAMPLES.append(x[:12])
print VCF_SAMPLES

['CPCT02010267', 'CPCT02010269', 'CPCT02010272', 'CPCT02010299', 'CPCT02010306', 'CPCT02010307', 'CPCT02010313', 'CPCT02010326', 'CPCT02010327', 'CPCT02020217', 'CPCT02020234', 'CPCT02020257']


<h3> LOAD VCF

In [4]:
#Load all VCF_SAMPLES to DF
df = pd.DataFrame({})
for VCF_SAMPLE in VCF_SAMPLES:
    VCF_FILE_NAME = VCF_SAMPLE + "R_"+ VCF_SAMPLE + "T_merged_somatics.vcf"
    SAMPLE_NAMES = {VCF_SAMPLE + 'T.mutect':'mutect', \
                    VCF_SAMPLE + 'T.freebayes':'freebayes', \
                   'TUMOR.strelka':'strelka', \
                   'TUMOR.varscan':'varscan'}
    df = pd.concat([df,aVCF.loadVaraintsFromVCF(VCF_PATH,VCF_FILE_NAME,SAMPLE_NAMES,VCF_SAMPLE)])

reading vcf file: CPCT02010267R_CPCT02010267T_merged_somatics.vcf
Number variants loaded: 44651
reading vcf file: CPCT02010269R_CPCT02010269T_merged_somatics.vcf
reading VCF File line: 100000
Number variants loaded: 105189
reading vcf file: CPCT02010272R_CPCT02010272T_merged_somatics.vcf
reading VCF File line: 100000
Number variants loaded: 130305
reading vcf file: CPCT02010299R_CPCT02010299T_merged_somatics.vcf
Number variants loaded: 83224
reading vcf file: CPCT02010306R_CPCT02010306T_merged_somatics.vcf
Number variants loaded: 69901
reading vcf file: CPCT02010307R_CPCT02010307T_merged_somatics.vcf
Number variants loaded: 54815
reading vcf file: CPCT02010313R_CPCT02010313T_merged_somatics.vcf
Number variants loaded: 38797
reading vcf file: CPCT02010326R_CPCT02010326T_merged_somatics.vcf
Number variants loaded: 66724
reading vcf file: CPCT02010327R_CPCT02010327T_merged_somatics.vcf
Number variants loaded: 65940
reading vcf file: CPCT02020217R_CPCT02020217T_merged_somatics.vcf
Number v

<h3> CHECKS

In [8]:
df[(df.variantType == "INDEL") & (df.variantSubType != "INSERT") & (df.variantSubType != "DELETE")]

,chrom,pos,chromPos,chromFrac,ref,vennSegment,numCallers,variantType,variantSubType,strelkaallele,strelkaallelicFreq,varscanallele,varscanallelicFreq,mutectallele,mutectallelicFreq,freebayesallele,freebayesallelicFreq,patientName
8004,3,83519247,383519247,3.421767,TGG,freebayes,1,INDEL,,,,,,,,TG,1,CPCT02010267
15999,6,112894453,6112894453,6.659758,TTAAT,freebayes,1,INDEL,,,,,,,,TTT,1,CPCT02010267
440,1,9530118,19530118,1.038235,AGG,freebayes,1,INDEL,,,,,,,,AG,0.964286,CPCT02010269
42381,7,29319999,729319999,7.184242,CATAA,freebayes,1,INDEL,,,,,,,,CAATAA,1,CPCT02010269
56539,9,89648759,989648759,9.634846,GAA,freebayes,1,INDEL,,,,,,,,GAAAGA,0.791667,CPCT02010269
60518,10,56222236,1056222236,10.414818,CTTTCTTTTCTTTTCTTTTCTT,freebayes,1,INDEL,,,,,,,,CTTTCTTTTCTTTTCTTTTCT,0.971831,CPCT02010269
79009,15,25975502,1525975502,15.253342,AGG,freebayes,1,INDEL,,,,,,,,AGGAAG,1,CPCT02010269
95674,22,16876861,2216876861,22.328954,CAAAAAAAAAAAG,freebayes,1,INDEL,,,,,,,,CAAAAAAAAAAAAG,0.959596,CPCT02010269
97200,X,6444663,X6444663,23.041506,AAGGAAGGAAGGAAGGA,freebayes,1,INDEL,,,,,,,,AAGGAAGGAAGGAAGGAA,0.923077,CPCT02010269
253,1,2789487,12789487,1.011191,TTCT,freebayes,1,INDEL,,,,,,,,TTCTTT,0.846154,CPCT02010272


In [6]:
len(df)

852956

<h3> High Level Stats

In [ ]:
pd.pivot_table(df, values='pos', index=['patientName'], columns=['variantType','numCallers'], aggfunc='count')

In [ ]:
#INTERNAL 'PRECISION' + 'SENSITIVITY' per caller
outputdata = []
variantTypes = df.variantType.unique()
patientNames = df.patientName.unique()
for patientName in patientNames:
    for variantType in variantTypes:
        truthSet = len(df[(df['numCallers'] > 0)&(df['variantType'] == variantType)&(df['patientName'] == patientName)])
        for columnName in list(df):
            if columnName.endswith('allele'):
                myCaller = columnName[:-6]
                truePositives = len(df[(df[myCaller+'allele'] != '')&(df['numCallers'] > 0) &\
                                   (df['variantType'] == variantType)&(df['patientName'] == patientName)])
                falseNegatives = len(df[(df[myCaller+'allele'] == '')&(df['numCallers'] > 0) &\
                                    (df['variantType'] == variantType)&(df['patientName'] == patientName)])
                positives = truthSet - falseNegatives 
                if positives > 0:
                    outputdata.append([patientName,variantType, myCaller, truthSet,truePositives,\
                                   positives-truePositives, falseNegatives, \
                                   round(truePositives/float(positives),4),round(truePositives/float(truthSet),4)])
   
outputDF = pd.DataFrame(outputdata)
outputDF.columns = (['patientName','variantType','caller','truthSet','truePositives','falsePositives','falseNegatives',\
                     'internalPrecision','internalSensitivity'])
outputDF.sort_values(['patientName','variantType','caller']).head(8)

In [ ]:
df[['variantType','pos']].groupby(['variantType']).agg('count')

<h3> SNP Venn

In [ ]:
snpdf = df[(df.variantType == 'SNP')]

In [ ]:
snpdf[['numCallers','vennSegment','pos']].groupby(['numCallers','vennSegment']).agg('count')

In [ ]:
vn.venn([snpdf[snpdf.mutectallele != '']['chromPos'], \
         snpdf[snpdf.strelkaallele != '']['chromPos'], \
        snpdf[snpdf.freebayesallele != '']['chromPos'], \
        snpdf[snpdf.varscanallele != '']['chromPos'] \
        ],['mutect','strelka','freebayes','varscan'],figsize=(6,6))

### <hd1> Indel Venn

In [ ]:
indeldf= df[(df.variantType == 'INDEL')]

In [ ]:
indeldf[['numCallers','vennSegment','pos']].groupby(['numCallers','vennSegment']).agg('count')

In [ ]:
vn.venn([indeldf[indeldf.strelkaallele != '']['chromPos'], \
        indeldf[indeldf.freebayesallele != '']['chromPos'], \
        indeldf[indeldf.varscanallele != '']['chromPos'] \
        ],['strelka','freebayes','varscan'],figsize=(6,6))

<h3> Allelic Depth

In [ ]:
#Alllele Freq By Caller
for columnName in list(snpdf):
    if columnName.endswith('allele'):
        ser = snpdf[snpdf[columnName] != ''][columnName[:-6] + 'allelicFreq']
        ser = ser.sort_values()
        ser[len(ser)] = ser.iloc[-1]
        cum_dist = np.linspace(0.,1.,len(ser))
        ser_cdf = pd.Series(cum_dist, index=ser,name=columnName[:-6]+": c="+str(ser.count())+" m="+str(round(ser.median(),2)))
        ser_cdf.plot(drawstyle='steps',legend=True,title=" Allelic Frequency by Caller",figsize=[15,6])

In [ ]:
# BY CALLER BY VENN SEGMENT - Single Caller
myCaller = 'freebayes'
vennSegments = snpdf[snpdf[myCaller+'allele'] != ''].vennSegment.unique()
for vennSegment in vennSegments:
    ser = snpdf[(snpdf.vennSegment == vennSegment) & (snpdf[myCaller+'allele'] != '')][myCaller+'allelicFreq']
    ser = ser.sort_values()
    ser[len(ser)] = ser.iloc[-1]
    cum_dist = np.linspace(0.,1.,len(ser))
    ser_cdf = pd.Series(cum_dist, index=ser,name=vennSegment+": c="+str(ser.count())+" m="+str(round(ser.median(),2)))
    ser_cdf.plot(drawstyle='steps',legend=True,title=myCaller+" by Venn Segment",figsize=[15,6])

In [ ]:
# BY CALLER BY VENN SEGMENT - Single Venn Segment
myVennSegment = 'strelka-freebayes-mutect'
for columnName in list(snpdf):
    if columnName.endswith('allele') and ((columnName[:-6] in myVennSegment) or (myVennSegment == 'Intersection')):
        ser = snpdf[(snpdf[columnName] != '') & (snpdf.vennSegment == myVennSegment)][columnName[:-6] + 'allelicFreq']
        ser = ser.sort_values()
        ser[len(ser)] = ser.iloc[-1]
        cum_dist = np.linspace(0.,1.,len(ser))
        ser_cdf = pd.Series(cum_dist, index=ser,name=columnName[:-6]+": c="+str(ser.count())+" m="+str(round(ser.median(),2)))
        ser_cdf.plot(drawstyle='steps',legend=True,title=" Allelic Frequency by Caller for Venn Segment: " \
                     +myVennSegment,figsize=[15,6])

In [ ]:
#Alllele Freq By Caller PRIVATE
for columnName in list(snpdf):
    if columnName.endswith('allele'):
        ser = snpdf[(snpdf[columnName] != '') & (snpdf.numCallers == 1)][columnName[:-6] + 'allelicFreq']
        ser = ser.sort_values()
        ser[len(ser)] = ser.iloc[-1]
        cum_dist = np.linspace(0.,1.,len(ser))
        ser_cdf = pd.Series(cum_dist, index=ser,name=columnName[:-6]+": c="+str(ser.count())+" m="+str(round(ser.median(),2)))
        ser_cdf.plot(drawstyle='steps',legend=True,title=" Allelic Frequency by Caller - Private Calls",figsize=[15,6])

### Allelic Depth Scatter Plot

In [ ]:
caller1 = 'varscan'
caller2 = 'strelka'
tempdf = df[(df[caller1+'allele']!='')&(df[caller2+'allele']!='')]
tempdf.head()
tempdf.plot.scatter(caller1+'allelicFreq',caller2+'allelicFreq' \
                        ,figsize=[6,6] \
                        ,title="Comparison of "+caller1+" and "+caller2+" Allelic Frequency" \
                        ,xlim=[0,1],ylim=[0,1])

<h3> RAINBOW CHARTS

In [ ]:
## RAINBOW
#plt.scatter(df[df.caller == 'mutect'].chromFrac, df[df.caller == 'mutect'].allelicFreq, s=10, c='b', marker="s")